In [ ]:
############ LOAD in custom packages ################
import sys
import os
import pandas as pd
import numpy as np
import importlib
import multiprocessing as mp
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import xarray as xr


project_dir = '/Users/demo/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2'
sub_logs = os.path.join(project_dir, 'subject_logs')

subs = {}
for sub in os.listdir(sub_logs):
    if 'sub-' in sub:
        parts = sub.split('.')
        sub_id = parts[0]
        subs[sub_id] = pd.read_excel(os.path.join(sub_logs, sub),sheet_name='JSON_formatted')



In [198]:
px.defaults.height=600
df = pd.concat([sub[['key','value']].T for sub in subs.values()], axis=0, ignore_index=True)
df.columns = df.iloc[0]
df = df[df['id'] != 'id']
heatmap_columns = [col for col in df.columns if not 'notes' in col and not 'diagnosis' in col]
#heatmap_columns.remove('subject_id')
#display(df[heatmap_columns])
xr_data = xr.DataArray(df[heatmap_columns].values, 
                       dims=["rows", "columns"],
                       coords={"rows": df['id'], "columns": heatmap_columns})
xr_data_bin = xr_data.where(xr_data == 0, 1)
#print(xr_data_bin)
fig = px.imshow(xr_data_bin, origin='lower',
                    title="Tasks Completed by Participants",
                    zmin=0, zmax=1, color_continuous_scale=[[0, "white"], [1, "black"]],
                    labels={ # replaces default labels by column name
                    "rows": "Subject",  "columns": "Tasks Completed"}
                    )
fig.show()
fig.write_html("dashboard.html") # interactive export 


In [ ]:
mindlamp_data = os.path.join(project_dir, 'mindlamp_data','processed_data')

accel = {}
for sub in os.listdir(mindlamp_data):
    if os.path.exists(os.path.join(mindlamp_data,sub,'processed','accel')):
        for file in os.listdir(os.path.join(mindlamp_data,sub,'processed','accel')):
            if 'phone_accel_activity' in file:
                accel[sub] = pd.read_csv(os.path.join(mindlamp_data,sub,'processed','accel',file))
                accel[sub]['subject_id'] = sub

activityScores = [col for col in df.columns if 'activityScore' in col]
accel_df = pd.concat([sub for sub in accel.values()], axis=0, ignore_index=True)


In [168]:
sub = 'PCT929'
df = accel_df.query("subject_id == @sub")
xr_accel = xr.DataArray(df[activityScores].values, 
                       dims=["rows", "columns"],
                       coords={"rows": df['day'], "columns": activityScores})
fig = px.imshow(xr_accel, origin='lower', title='Activity Scores for PCT927',
                zmin=0,zmax=5,height=400, width=600)
fig.show()

In [200]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Input, Output

app = Dash()
df = accel_df

# App layout
app.layout = [
    html.Div(children='Accelerometer Data'),
    dcc.Dropdown(id="subject_id", value='PCT927', clearable=False,
        options=[{"label": f'sub-{y}', "value": y} for y in df['subject_id'].unique()]),
    dcc.Graph(figure={}, id='controls-and-graph')
]

# Add controls to build the interaction
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='subject_id', component_property='value')
)


def cb(subject_id):
    """Callback to update the figure based on the selected id"""
    sub = subject_id
    df = accel_df.query("subject_id == @sub")
    xr_accel = xr.DataArray(df[activityScores].values, 
                        dims=["rows", "columns"],
                        coords={"rows": df['day'], "columns": activityScores})
    fig = px.imshow(xr_accel, origin='lower', title=f'Activity Scores for sub-{sub}',
                    zmin=0,zmax=5,height=400, width=600)
    return fig

if __name__ == '__main__':
    app.run(mode='inline',port=8052)  # or 'external' if you want it in a new tab

In [245]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Input, Output

app = Dash()

mindlamp_data = os.path.join(project_dir, 'mindlamp_data','processed_data')

power = {}
for sub in os.listdir(mindlamp_data):
    if os.path.exists(os.path.join(mindlamp_data,sub,'processed','power')):
        for file in os.listdir(os.path.join(mindlamp_data,sub,'processed','power')):
            if 'phone_power_activityScores_hourly' in file:
                power[sub] = pd.read_csv(os.path.join(mindlamp_data,sub,'processed','power',file))
                power[sub]['subject_id'] = sub

activityScores = [col for col in df.columns if 'activityScore' in col]
power_df = pd.concat([sub for sub in power.values()], axis=0, ignore_index=True)

df = power_df

# App layout
app.layout = [
    html.Div(children='Phone Use Data'),
    dcc.Dropdown(id="subject_id", value='PCT927', clearable=False,
        options=[{"label": f'sub-{y}', "value": y} for y in df['subject_id'].unique()]),
    dcc.RadioItems(id='days', value='percentHome', 
        options=['All days','Weekdays','Weekends']),

    dcc.Graph(figure={}, id='controls-and-graph')
]

# Add controls to build the interaction
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='subject_id', component_property='value'),
    Input(component_id='days', component_property='value')
)


def cb(subject_id, days):
    """Callback to update the figure based on the selected id"""
    sub = subject_id
    df = power_df.query("subject_id == @sub")
    if days == 'Weekdays':
        df = df[df['weekday'].astype(str).str.contains('1|2|3|4|5')]
    elif days == 'Weekends':
        df = df[df['weekday'].astype(str).str.contains('6|7')]
    xr_power = xr.DataArray(df[activityScores].values, 
                        dims=["rows", "columns"],
                        coords={"rows": df['day'], "columns": activityScores})
    fig = px.imshow(xr_power, origin='lower', title=f'Phone Activity (minutes each hour) for sub-{sub}',
                    zmin=0,zmax=60,height=400, width=600)
    return fig

if __name__ == '__main__':
    app.run(mode='inline',port=8053)  # or 'external' if you want it in a new tab

In [219]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Input, Output

app = Dash()

mindlamp_data = os.path.join(project_dir, 'mindlamp_data','processed_data')

gps = {}
for sub in os.listdir(mindlamp_data):
    if os.path.exists(os.path.join(mindlamp_data,sub,'processed','gps')):
        for file in os.listdir(os.path.join(mindlamp_data,sub,'processed','gps')):
            if 'phone_gps_freq_hourly' in file:
                gps[sub] = pd.read_csv(os.path.join(mindlamp_data,sub,'processed','gps',file))
                gps[sub]['subject_id'] = sub

activityScores = [col for col in df.columns if 'activityScore' in col]
gps_df = pd.concat([sub for sub in gps.values()], axis=0, ignore_index=True)

df = gps_df

# App layout
app.layout = [
    html.Div(children='GPS Data'),
    dcc.Dropdown(id="subject_id", value='PCT927', clearable=False,
        options=[{"label": f'sub-{y}', "value": y} for y in df['subject_id'].unique()]),
    dcc.Graph(figure={}, id='controls-and-graph')
]

# Add controls to build the interaction
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='subject_id', component_property='value')
)


def cb(subject_id):
    """Callback to update the figure based on the selected id"""
    sub = subject_id
    df = gps_df.query("subject_id == @sub")
    xr_gps = xr.DataArray(df[activityScores].values, 
                        dims=["rows", "columns"],
                        coords={"rows": df['day'], "columns": activityScores})
    fig = px.imshow(xr_gps, origin='lower', title=f'GPS Activity per Hour for sub-{sub}',
                    zmin=0,zmax=5,height=400, width=600)
    return fig

if __name__ == '__main__':
    app.run(mode='inline',port=8053)  # or 'external' if you want it in a new tab

In [267]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Input, Output

app = Dash()

mindlamp_data = os.path.join(project_dir, 'mindlamp_data','processed_data')

gps = {}
for sub in os.listdir(mindlamp_data):
    if os.path.exists(os.path.join(mindlamp_data,sub,'processed','gps')):
        for file in os.listdir(os.path.join(mindlamp_data,sub,'processed','gps')):
            if 'phone_gps_homeStay_daily' in file:
                gps[sub] = pd.read_csv(os.path.join(mindlamp_data,sub,'processed','gps',file))
                gps[sub]['subject_id'] = sub

gps_df = pd.concat([sub for sub in gps.values()], axis=0, ignore_index=True)

# App layout
app.layout = [
    html.Div(children='Mobility Information for Participants'),
    dcc.Dropdown(id="subject_id", value='PCT927', clearable=False,
        options=[{"label": f'sub-{y}', "value": y} for y in df['subject_id'].unique()]),
    dcc.RadioItems(id='gps_item', value='percentHome', 
        options=['radiusMobility','percentHome','numPlaces']),
    dcc.Graph(figure={}, id='controls-and-graph')
]

# Add controls to build the interaction
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='subject_id', component_property='value'),
    Input(component_id='gps_item', component_property='value')
)


def cb(subject_id, gps_item):
    """Callback to update the figure based on the selected id"""
    sub = subject_id
    sub_df = gps_df.query("subject_id == @sub")
    fig = px.line(sub_df, x="day", y=gps_item, title=f'{gps_item} for sub-{sub}')
    return fig

if __name__ == '__main__':
    app.run(mode='inline',port=8055)  # or 'external' if you want it in a new tab

In [269]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Input, Output
import dash_bootstrap_components as dbc
activity_app = Dash()
mindlamp_data = os.path.join(project_dir, 'mindlamp_data','processed_data')

accel = {}
for sub in os.listdir(mindlamp_data):
    if os.path.exists(os.path.join(mindlamp_data,sub,'processed','accel')):
        for file in os.listdir(os.path.join(mindlamp_data,sub,'processed','accel')):
            if 'phone_accel_activity' in file:
                accel[sub] = pd.read_csv(os.path.join(mindlamp_data,sub,'processed','accel',file))
                accel[sub]['subject_id'] = sub
accel_df = pd.concat([sub for sub in accel.values()], axis=0, ignore_index=True)

activityScores = [col for col in accel_df.columns if 'activityScore' in col]

gps = {}
for sub in os.listdir(mindlamp_data):
    if os.path.exists(os.path.join(mindlamp_data,sub,'processed','gps')):
        for file in os.listdir(os.path.join(mindlamp_data,sub,'processed','gps')):
            if 'phone_gps_homeStay_daily' in file:
                gps[sub] = pd.read_csv(os.path.join(mindlamp_data,sub,'processed','gps',file))
                gps[sub]['subject_id'] = sub

gps_df = pd.concat([sub for sub in gps.values()], axis=0, ignore_index=True)

# App layout
activity_app.layout = dbc.Container[
    html.H1(children='Accelerometer Data'),
    dcc.Dropdown(id='subject_id_input', value='PCT927', clearable=False,
        options=[{"label": f'sub-{y}', "value": y} for y in accel_df['subject_id'].unique()]),
    dcc.Graph(figure={}, id='accel_graph')
    
    # html.Div(children='Mobility Information for Participants'),
    # dcc.Dropdown(id="subject_id_input", value='PCT927', clearable=False,
    #     options=[{"label": f'sub-{y}', "value": y} for y in gps_df['subject_id'].unique()]),
    # dcc.RadioItems(id='gps_item', value='percentHome', 
    #     options=['radiusMobility','percentHome','numPlaces']),
    # dcc.Graph(figure={}, id='gps-graph')
]

# Add controls to build the interaction
@callback(
    Output(component_id='accel_graph', component_property='figure'),
    #Output(component_id='gps-graph', component_property='figure'),
    Input(component_id='subject_id_input', component_property='value'),
    Input(component_id='gps_item', component_property='value')
)


def cb(subject_id_input, gps_item):
    """Callback to update the figure based on the selected id"""
    sub = subject_id_input
    accel_sub = accel_df.query("subject_id == @sub")
    accel_xr = xr.DataArray(accel_sub[activityScores].values, 
                        dims=["rows", "columns"],
                        coords={"rows": accel_sub['day'], "columns": activityScores})
    accel_fig = px.imshow(accel_xr, origin='lower', title=f'Activity Scores for sub-{sub}',
                    zmin=0,zmax=5,height=400, width=600)
    
    # gps_sub = gps_df.query("subject_id == @sub")
    # gps_fig = px.line(gps_sub, x="day", y=gps_item, title=f'{gps_item} for sub-{sub}')

    return accel_fig


if __name__ == '__main__':
    activity_app.run(mode='inline',port=8052)  # or 'external' if you want it in a new tab

TypeError: type 'Container' is not subscriptable

In [ ]:
# Initialize the app - incorporate css
import dash_bootstrap_components as dbc

external_stylesheets = [dbc.themes.CERULEAN]
gps_app = Dash(__name__, external_stylesheets=external_stylesheets)

gps = {}
for sub in os.listdir(mindlamp_data):
    if os.path.exists(os.path.join(mindlamp_data,sub,'processed','gps')):
        for file in os.listdir(os.path.join(mindlamp_data,sub,'processed','gps')):
            if 'phone_gps_homeStay_daily' in file:
                gps[sub] = pd.read_csv(os.path.join(mindlamp_data,sub,'processed','gps',file))
                gps[sub]['subject_id'] = sub

gps_df = pd.concat([sub for sub in gps.values()], axis=0, ignore_index=True)

# App layout
gps_app.layout = [
    html.Div(children='Mobility Information for Participants'),
    dcc.Dropdown(id="subject_id", value='PCT927', clearable=False,
        options=[{"label": f'sub-{y}', "value": y} for y in df['subject_id'].unique()]),
    dcc.RadioItems(id='gps_item', value='radiusMobility', 
        options=['radiusMobility','percentHome','numPlaces']),
    dcc.Graph(figure={}, id='controls-and-graph')
]

# Add controls to build the interaction
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='subject_id', component_property='value'),
    Input(component_id='gps_item', component_property='value')
)


def cb(subject_id, gps_item):
    """Callback to update the figure based on the selected id"""
    sub = subject_id
    gps_sub = gps_df.query("subject_id == @sub")
    gps_fig = px.line(gps_sub, x="day", y=gps_item, title=f'{gps_item} for sub-{sub}')
    return gps_fig

if __name__ == '__main__':
    gps_app.run(mode='inline',port=8054)  # or 'external' if you want it in a new tab